In [3]:
# External libraries
import requests

# Builtins
import json
import os
import concurrent.futures
import functools
import zipfile
import pathlib

In [4]:
def get_json(overwrite=False):
    """
    return a dictionary which contains all pokemon urls
    """
    res_json = None
    path = fr"./data/pokemon.json"
    if overwrite or not os.path.exists(path):
        offset, limit = 0, 10000
        url = fr"https://pokeapi.co/api/v2/pokemon/?offset={offset}&limit={limit}/"
        headers = {'user-agent': 'pokemon-gallery'}
        res_json = requests.get(url, headers = headers).json()
        with open(path, "w", encoding="utf8") as f:
            json.dump(res_json, f)
    else:
        with open(path, "r", encoding="utf8") as f:
            res_json = json.load(f)
    return res_json

In [5]:
def download_pokemon_detail(info, overwrite=False):
    """
    """
    id, name = info
    path = fr"./data/pokemon_details_pokeapi/{id}_{name}.json"
    if overwrite or not os.path.exists(path):
        url = fr"https://pokeapi.co/api/v2/pokemon/{id}/"
        headers = {'user-agent': 'pokemon-gallery'}
        res_json = requests.get(url, headers = headers).json()
        with open(path, "w", encoding="utf8") as f:
            json.dump(res_json, f)
    else:
        with open(path, "r", encoding="utf8") as f:
            res_json = json.load(f)
    return res_json


### Example of how `res_json` looks
```python
{'count': 964,
 'next': None,
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  ...]
}
```

In [11]:
# Make directries (recursively) if not exist.
pathlib.Path("./data/pokemon_details_pokeapi/").mkdir(parents=True, exist_ok=True)

# Get all pokemon IDs
res_json = get_json()
infos = [(pokemon['url'].rsplit("/", 2)[-2], pokemon["name"]) for pokemon in res_json["results"]]

# Multithreading
no_threads = 10
with concurrent.futures.ThreadPoolExecutor(max_workers=no_threads) as executor:
    partial_func = functools.partial(download_pokemon_detail, overwrite=False)
    executor.map(partial_func, infos)

### Make a zip file for archiving

In [15]:
path = "./data/pokemon_details_pokeapi/"
with zipfile.ZipFile(f'{path[:-1]}.zip','w') as zip_file: 
    for file in os.listdir(f"{path}"):
        zip_file.write(f"{path}{file}", f"{file}", compress_type=zipfile.ZIP_DEFLATED)
print(f"Zip to {path[:-1]}.zip successfully")

Zip to ./data/pokemon_details_pokeapi.zip successfully
